# Topic modeling

Zadanie proszę wykonać w parach, ale... na dwóch komputerach. Na obu komputerach proszę najpierw wykonać instrukcje z sekcji **Przygotowanie**. Następnie na jednym komputerze proszę od razu odkomentować i uruchomić kod w sekcji **Wikipedia** i **Wizualizacja**. Na drugim komputerze proszę rozwiązać zadania z sekcji **Tweety** i **Wizualizacja**.

Po wykonaniu tego zadania powinieneś:
+ potrafić wykonać podstawowy topic modeling,
+ umieć stworzyć słownik mapujący identyfikatory na słowa,
+ potrafić stworzyć macierz wektorów TF-IDF,
+ wiedzieć jak wykorzystać LDA do określenia proporcji tematów w nowym dokumencie tekstowym,
+ potrafić zwizualizować wyniki algorytmu LDA.

Do wykonania zadania wykorzystamy bibliotekę [gensim](https://radimrehurek.com/gensim/), która oferuje szereg metod do analizy tekstu. Warto w wolnej chwili zobaczyć co oprócz algorytmu LDA zostało zaimplementowane w ramach tego modułu!

## Przygotowanie

Aby zadziałała wizualizacja, musimy najpierw zauktualizować bibliotekę `scipy` i doinstalować bibliotekę `pyldavis`. Będzie to okazja, żeby zobaczyć jak zarządza się bibliotekami w anacondzie. Jeśli kogoś interesuje co oznaczają kolejne komendy, proszę zajrzeć do [dokumentacji Anacondy](http://conda.pydata.org/docs/using/pkgs.html).

1. Zatrzymaj kernel (serwer jupyter notebooka)
2. Otwórz terminal
3. Wpisz `conda update scipy` (i Enter gdy spytają `Proceed`)
4. Popatrz na paski postępu
5. Wpisz `activate root`
6. Wpisz `pip install pyldavis`
7. Wpisz `deactivate`

Koniec. Możesz ponownie odpalić notatnik i przejść do kolejnych kroków.

## Wikipedia

Ten fragemnt kodu stanowi przykład uruchomienia topic modelingu na większym zbiorze danych. Ponieważ wyliczenie modelu będzie trwać od kilku do kilkunastu minut, niech każda para uruchomi ten przykład tylko na jednym komputerze.

Aby uruchomić przykład, w folderze z notatnikiem muszą znajdować się pliki `wiki_wordids.txt.bz2` i `wiki_tfidf.mm` ściągnięte wraz z notatnikiem. Przykład zbudowany jest w oparciu o podzbiór stron wikipedii dostępny pod adresem: https://dumps.wikimedia.org/enwiki/latest/. Strony zostały przekonwertowane na reprezentację wektorową za pomocą skryptu:
`python -m gensim.scripts.make_wiki`.

**Przeczytaj komentarze zanim uruchomisz kod.**

In [1]:
import logging
import gensim
import warnings
warnings.filterwarnings("ignore")

# Włączamy logowanie, żeby śledzić postępy algorytmu (to akurat nie będzie działać w Jupyter Notebooku,
# ale warto o tym wspomnieć)
# logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

# Odczytujemy z pliku mapowanie/słownik id->słowo
id2word = gensim.corpora.Dictionary.load_from_text('wiki_wordids.txt.bz2')

# Odczytujemy z pliku reprezentację wektorową korpusu (macierz wetkorów TF-IDF)
mm = gensim.corpora.MmCorpus('wiki_tfidf.mm')
print(mm)

# Tworzymy model LDA z 20 grupami wykonując 20 iteracji na całym zbiorze
lda = gensim.models.LdaMulticore(corpus=mm, id2word=id2word, num_topics=20, passes=20, workers=4)
# Alternatywnie w razie problemów z wielowątkowością:
# lda = gensim.models.ldamodel.LdaModel(corpus=mm, id2word=id2word, num_topics=20, update_every=0, passes=20)
lda.print_topics(20)

MmCorpus(4672 documents, 19960 features, 1762996 non-zero entries)


[(0,
  '0.002*"canis" + 0.002*"concord" + 0.002*"aberration" + 0.002*"chord" + 0.001*"aston" + 0.001*"peirce" + 0.001*"markov" + 0.001*"clutch" + 0.001*"butt" + 0.001*"skinner"'),
 (1,
  '0.004*"angola" + 0.002*"batting" + 0.002*"angolan" + 0.002*"braille" + 0.001*"appellate" + 0.001*"edvard" + 0.001*"interpol" + 0.001*"andrei" + 0.001*"unctad" + 0.001*"upu"'),
 (2,
  '0.004*"album" + 0.003*"acid" + 0.003*"carbon" + 0.002*"cells" + 0.002*"players" + 0.002*"cell" + 0.002*"disease" + 0.002*"songs" + 0.002*"chemistry" + 0.002*"protein"'),
 (3,
  '0.005*"comoros" + 0.003*"antarctic" + 0.003*"samoa" + 0.002*"ajax" + 0.002*"fm" + 0.002*"bahá" + 0.002*"antarctica" + 0.002*"lm" + 0.002*"cardinality" + 0.002*"odysseus"'),
 (4,
  '0.007*"alfonso" + 0.005*"afonso" + 0.004*"albert" + 0.004*"charlemagne" + 0.003*"abd" + 0.003*"ambrose" + 0.003*"basque" + 0.002*"esperanto" + 0.002*"rahman" + 0.002*"castile"'),
 (5,
  '0.003*"cuisine" + 0.002*"sauce" + 0.002*"amiga" + 0.002*"babbage" + 0.002*"dishes"

## Tweety

W parach stwórzcie model LDA w oparciu o załączone Tweety. W tym celu należy przekonwertować pliki tekstowe na reprezentację wektorową zapisując wcześniej mapowanie id->słowo w postaci słownika. Opis jak stworzyć wymienione struktury danych można znaleźć na stronie: https://radimrehurek.com/gensim/tut1.html.

**Zad. 1: Wczytaj tweety z pliku tweets.tsv do zmiennej `tweets`.**

In [2]:
import gensim
import logging
import nltk
import re
import pandas as pd

stopwords = ["a", "about", "after", "all", "am", "an", "and", "any", "are", "as", "at", "be", "because", "been",
            "before", "being", "between", "both", "by", "could", "did", "do", "does", "doing", "during", "each",
            "for", "from", "further", "had", "has", "have", "having", "he", "he'd", "he'll", "he's", "her", "here",
            "here's", "hers", "herself", "him", "himself", "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've",
            "in", "into", "is", "it", "it's", "its", "itself", "let's", "me", "more", "most", "my", "myself", "of",
            "on", "once", "only", "or", "other", "ought", "our", "ours", "ourselves", "own", "shan't", "she", "she'd",
            "she'll", "she's", "should", "so", "some", "such", "than", "that", "that's", "the", "their", "theirs",
            "them", "themselves", "then", "there", "there's", "these", "they", "they'd", "they'll", "they're",
            "they've", "this", "those", "through", "to", "until", "up", "very", "was", "wasn't", "we", "we'd",
            "we'll", "we're", "we've", "were", "weren't", "what", "what's", "when", "when's", "where", "where's",
            "which", "while", "who", "who's", "whom", "with", "would", "you", "you'd", "you'll", "you're", "you've",
            "your", "yours", "yourself", "yourselves", "above", "again", "against", "aren't", "below", "but", "can't",
            "cannot", "couldn't", "didn't", "doesn't", "don't", "down", "few", "hadn't", "hasn't", "haven't", "if",
            "isn't", "mustn't", "no", "nor", "not", "off", "out", "over", "shouldn't", "same", "too", "under", "why",
            "why's", "won't", "wouldn't"]

tweets = pd.read_csv('tweets.tsv', sep='\t', header=None)[2]

**Zad. 2: Dokonaj tokenizacji słów, usuń te z stoplisty (`stopwords`) oraz występujące tylko raz. Wynik przypisz do zmiennej `texts`.**

In [3]:
RE_SPACES = re.compile("\s+")
RE_HASHTAG = re.compile("[@#][_a-z0-9]+")
RE_EMOTICONS = re.compile("(:-?\))|(:p)|(:d+)|(:-?\()|(:/)|(;-?\))|(<3)|(=\))|(\)-?:)|(:'\()|(8\))")
RE_HTTP = re.compile("http(s)?://[/\.a-z0-9]+")

class BeforeTokenizationNormalizer():
    @staticmethod
    def normalize(text):
        text = text.strip().lower()
        text = text.replace('&nbsp;', ' ')
        text = text.replace('&lt;', '<')
        text = text.replace('&gt;', '>')
        text = text.replace('&amp;', '&')
        text = text.replace('&pound;', u'£')
        text = text.replace('&euro;', u'€')
        text = text.replace('&copy;', u'©')
        text = text.replace('&reg;', u'®')
        return text

class TweetTokenizer():
    @staticmethod
    def tokenize(text):
        tokens = re.split(RE_SPACES, text)
        output_tokens = []
        i = 0
        while i < len(tokens):
            token = tokens[i]
            match = []
            for j in [RE_HASHTAG, RE_EMOTICONS, RE_HTTP]:
                is_match = j.match(token)
                if is_match:
                    match.append(is_match)
            if match:
                if match[0].re != RE_HTTP:
                    p, k = match[0].span()
                    output_tokens.append(token[p:k])
            else:
                output_tokens += nltk.word_tokenize(token)
            i += 1
        return output_tokens

texts_temp = []
flattened_texts = []

for i in tweets.index:
    tweet = BeforeTokenizationNormalizer.normalize(tweets.iat[i])
    # Tokenize
    words_nltk = TweetTokenizer.tokenize(tweet)
    # Remove stopwords
    words_nltk = [i for i in words_nltk if i not in stopwords]
    # Append words to texts
    texts_temp.append(words_nltk)
    flattened_texts.extend(words_nltk)

# Remove words taht appear only once
from collections import Counter
cnt = Counter(flattened_texts)
words_once = [k for k, v in cnt.items() if v == 1]
# Append words that occure more than once
texts = []
for doc in texts_temp:
    new_doc = []
    for word in doc:
        if word not in words_once:
            new_doc.append(word)
    if new_doc: texts.append(new_doc)

flattened_texts = [item for sublist in texts for item in sublist]

print(texts[:5])
print(len(flattened_texts))

[['dear', '@microsoft', 'mac', 'great', ',', 'update', '?', "c'mon", '.'], ['@microsoft', 'make', 'system', "n't", 'eat', '.', '2nd', 'time', 'happened', 'sick', '!'], ['available'], ['available'], ['make', 'game', '#windows10', 'app', '.', 'will', 'owners', 'able', 'download', 'play', 'november', '?', '@microsoft']]
72091


**Zad. 3: Stwórz słownik id->słowo i przypisz do zmiennej `id2word`.**

In [4]:
from gensim.corpora import Dictionary

id2word = Dictionary(texts)
print(id2word)

Dictionary(4934 unique tokens: [',', '.', '?', '@microsoft', "c'mon"]...)


**Zad. 4: Stwórz reprezentację wektorową korpusu (macierz wektorów TF-IDF), wynik przypisz do zmiennej `mm`.**

In [5]:
from gensim.models import TfidfModel

corpus = [id2word.doc2bow(text) for text in texts]
tfidf = TfidfModel(corpus)
mm = tfidf[corpus]
print(mm)

**Zad. 5: Odpal poniższy kod i odkryj 10 tematów za pomocą algorytmu LDA. Jeśli masz czas, zwiększ wartości parametrów num_topics i passes przy tworzeniu modelu LDA, i sprawdź jak to wpłynie na rezultat.**

In [6]:
# Włączamy logowanie, żeby śledzić postępy algorytmu (to akurat nie będzie działać w Jupyter Notebooku,
# ale warto o tym wspomnieć)
# logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

lda = gensim.models.ldamodel.LdaModel(corpus=mm, id2word=id2word, num_topics=10, update_every=0, passes=20)
# alternatywnie lda = gensim.models.LdaMulticore(corpus=mm, id2word=id2word, num_topics=10, passes=20)
lda.print_topics(10)

[(0,
  '0.010*"tomorrow" + 0.010*"." + 0.009*"," + 0.009*"!" + 0.007*"\'s" + 0.007*"\'m" + 0.007*"going" + 0.007*"ihop" + 0.007*"just" + 0.006*"like"'),
 (1,
  '0.009*"." + 0.008*"?" + 0.008*"church" + 0.008*"!" + 0.008*"eric" + 0.007*"," + 0.007*"\'s" + 0.007*"tomorrow" + 0.005*"night" + 0.004*"going"'),
 (2,
  '0.010*"galaxy" + 0.009*"note" + 0.008*"5" + 0.007*":" + 0.007*"," + 0.007*"samsung" + 0.006*"." + 0.006*"..." + 0.006*"may" + 0.005*"\'s"'),
 (3,
  '0.011*"!" + 0.007*"," + 0.007*"." + 0.005*"\'s" + 0.004*"may" + 0.004*"..." + 0.004*":" + 0.003*"?" + 0.003*"n\'t" + 0.003*"saturday"'),
 (4,
  '0.009*"." + 0.007*"," + 0.007*"?" + 0.007*"xxl" + 0.007*"mike" + 0.007*"federer" + 0.007*"magic" + 0.005*"may" + 0.005*":" + 0.004*"time"'),
 (5,
  '0.008*"," + 0.007*"." + 0.007*"!" + 0.005*"\'s" + 0.005*"mariah" + 0.005*"?" + 0.005*"carey" + 0.005*"may" + 0.005*"n\'t" + 0.005*":"'),
 (6,
  '0.009*"." + 0.007*"," + 0.006*"\'s" + 0.006*"may" + 0.006*"trump" + 0.005*"donald" + 0.005*":" + 

**Zad. 6*: Na podstawie zbudowanego modelu określ proporcje tematów w następującym tweecie:
`Zlatan is looking mighty attractive at the moment,if LVG doesn't get a striker by Tuesday, I really don't fancy us scoring goals this season`. Jeśli zostało niewiele czasu, przejdź od razu do wizualizacji.**

In [7]:
new_tweet = "Zlatan is looking mighty attractive at the moment,if LVG doesn't get a striker by Tuesday, I really don't fancy us scoring goals this season"
# Preprocess
new_tweet = BeforeTokenizationNormalizer.normalize(new_tweet)
new_tweet = TweetTokenizer.tokenize(new_tweet)
new_tweet_text = [i for i in new_tweet if i not in stopwords]
# Add as document
new_tweet_mm = id2word.doc2bow(new_tweet_text)
# Predict topics
new_tweet_lda = lda[new_tweet_mm]
print(new_tweet_lda)

[(7, 0.70549417), (9, 0.2411404)]


## Wizualizacja

Spróbujemy teraz zwizualizować uzyskane tematy. Podążaj za komentarzami a powinno się udać.

P.S. Możesz uruchomić tę wizualizację również dla tematów odkrytych z Wikipedii...

**Zad. 7: Zwizualizuj uzyskane tematy.**

In [8]:
# Poniższy kod korzysta modułu pyLDAvis z githuba:
# https://github.com/bmabey/pyLDAvis

import pyLDAvis
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

pyLDAvis.gensim.prepare(lda, mm, id2word)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.075062 -0.065216       1        1  15.968530
7     -0.051253 -0.068499       2        1  12.654865
9     -0.039067 -0.013930       3        1  11.087500
1     -0.030029  0.004258       4        1  10.535912
4     -0.021845  0.082669       5        1  10.369964
6     -0.009339  0.072854       6        1   9.051449
5     -0.024783  0.019583       7        1   9.003369
2     -0.004490 -0.016991       8        1   7.907906
3      0.000078  0.008448       9        1   7.888818
8      0.255790 -0.023175      10        1   5.531688, topic_info=           Term        Freq       Total Category  logprob  loglift
18    available  356.000000  356.000000  Default  30.0000  30.0000
2641     galaxy   22.000000   22.000000  Default  29.0000  29.0000
316        note   21.000000   21.000000  Default  28.0000  28.0000
2968      harry   20.000000   20.000000  Default  27.0000  27.0000
4192     potter   18.000000   18.000000  Default  26.0000  26.0000
...         ...         ...         ...      ...      ...      ...
0             ,    1.648569  133.004807  Topic10  -6.3810  -1.4958
102          's    1.207501   96.354664  Topic10  -6.6924  -1.4848
498        last    1.032159   23.722441  Topic10  -6.8493  -0.2401
1031       till    0.962588    6.758337  Topic10  -6.9191   0.9458
74          day    0.955802   59.899915  Topic10  -6.9261  -1.2432

[750 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
9         1  0.207993        !
9         2  0.174715        !
9         3  0.116476        !
9         4  0.116476        !
9         5  0.066558        !
...     ...       ...      ...
762       3  0.618324       ya
1471      2  0.480436      yay
3798      9  0.506865  youtube
3798     10  0.253433  youtube
2267      2  0.706672     zaza

[1743 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 8, 10, 2, 5, 7, 6, 3, 4, 9])